<a href="https://colab.research.google.com/github/anoushkagarg003/sentiment_analysis_amazon/blob/main/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.naive_bayes import MultinomialNB

In [ ]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.8 MB/s eta 0:00:00


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
def get_sentiment_label(text):
    scores = analyzer.polarity_scores(text)
    compound_score = scores['compound']
    if compound_score >= 0.05:
        return 'positive'
    elif compound_score <= -0.05:
        return 'negative'
    else:
        return 'neutral'
df['vader_sentiment']=df['clean_text_train'].apply(get_sentiment_label)

In [ ]:
df=pd.read_csv('clean_text_new.csv')
df['sentiment'] = df['Stars'].apply(lambda x: 'Positive' if x >= 3 else 'Negative')
df['clean_text_train'] = df['clean_text_train'].fillna('')

In [ ]:
X = df['clean_text_train']
y = df['vader_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(X_train)
tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

nb_classifier = MultinomialNB()

nb_classifier.fit(X_train_tfidf, y_train)
accuracy = nb_classifier.score(X_test_tfidf, y_test)
print("Accuracy:", accuracy)

6624    say water resistant not value money carry pair...
5664            inner stitching came inner stitching came
7757                              play video good product
6946                                       small not good
6686    kit bag small size zipper good strap adjustabl...
                              ...                        
5226                             good quality value money
5390                                    good product good
860                excellent bag office purpose excellent
7603    impressive ater absorption capability used tim...
7270    product received good packaging put transparen...
Name: clean_text_train, Length: 7258, dtype: object
Accuracy: 0.7125154894671624


In [ ]:
X = df['clean_text_train']  # Input features
y = df['vader_sentiment']  # Target variable

# Preprocessing: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Preprocessing: Vectorize the text data
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Model training: Linear Support Vector Classifier (SVC)
model = LinearSVC()
model.fit(X_train_vec,y_train)
y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

print('Accuracy:', accuracy)
print('Classification Report:')
print(classification_report_str)

Accuracy: 0.8623682579045258
Classification Report:
              precision    recall  f1-score   support

    negative       0.81      0.77      0.79       453
     neutral       0.76      0.73      0.75       176
    positive       0.90      0.93      0.92       984

    accuracy                           0.86      1613
   macro avg       0.82      0.81      0.82      1613
weighted avg       0.86      0.86      0.86      1613



In [ ]:
from sklearn.ensemble import RandomForestClassifier
X = df['clean_text_train']
y = df['vader_sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
model = RandomForestClassifier()
model.fit(X_train_vec, y_train)

y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.8828270303781773


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(X_train_vec, y_train)
y_pred = logreg.predict(X_test_vec)
print(f"Logistic Regression Accuracy: {accuracy_score(y_test, y_pred)}")


Logistic Regression Accuracy: 0.8400495970241786


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier()
gbc.fit(X_train_vec, y_train)
y_pred = gbc.predict(X_test_vec)
print(f"Gradient Boosting Classifier Accuracy: {accuracy_score(y_test, y_pred)}")

Gradient Boosting Classifier Accuracy: 0.8338499690018599


In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import torch
# Load the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# Define the function to predict sentiment
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    predicted_class_id = logits.argmax().item()
    return model.config.id2label[predicted_class_id]
df['new_sentiment'] = df['clean_text_train'].apply(predict_sentiment)

In [ ]:
df['new_sentiment']

0       POSITIVE
1       POSITIVE
2       POSITIVE
3       POSITIVE
4       POSITIVE
          ...   
8060    NEGATIVE
8061    POSITIVE
8062    NEGATIVE
8063    NEGATIVE
8064    NEGATIVE
Name: new_sentiment, Length: 8065, dtype: object

In [ ]:
df['sentiment'] = df['sentiment'].str.lower()
df['new_sentiment'] = df['new_sentiment'].str.lower()

# Calculate accuracy
accuracy = (df['sentiment'] == df['new_sentiment']).mean() * 100

print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 76.81%


In [36]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

def predict_sentiment(text):
    # Step 1: Load tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
    model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

    # Step 2: Tokenize input text
    inputs = tokenizer(text, return_tensors="pt")

    # Step 3: Perform inference
    with torch.no_grad():
        outputs = model(**inputs)

    # Step 4: Interpret the outputs
    predictions = outputs.logits.argmax(-1)
    predicted_label = tokenizer.decode(predictions[0])

    return predicted_label
df['new_sentiment'] = df['clean_text_train'].apply(predict_sentiment)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Streaming output truncated to the last 5000 lines.
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPre

KeyboardInterrupt: 